Imports

In [1]:
import numpy as np
import difflib
import pandas as pd
import unicodedata
import codecs
import math
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM
# from keras.layers.convolutional import Conv1D
# from keras.layers.convolutional import MaxPooling1D
# from keras.layers.embeddings import Embedding
# from keras.preprocessing import sequence
# from sklearn.model_selection import train_test_split, StratifiedKFold
# from sklearn.feature_extraction.text import CountVectorizer
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.models import Sequential
# from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# from keras.utils.np_utils import to_categorical
from fuzzywuzzy import fuzz

import re

ModuleNotFoundError: No module named 'numpy'

In [2]:
from IPython.display import display, clear_output
from itertools import islice

def next_n_lines(file_opened, N):
    return [x.split("\t") for x in islice(file_opened, N)]

In [3]:
dado_completo = pd.read_csv("DATA_LAUDOS_TEXTO_formato1", sep = ";")

In [4]:
frases_doenca = pd.read_csv("DicionarioECG.csv", sep = ";")
# frases_doenca
frases_doenca = frases_doenca[frases_doenca['laudado'] == 0]
del frases_doenca["laudado"]

In [5]:
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()

sample_100 = dado_completo.sample(1000)
print(len(dado_completo))

def checa_texto(index, txt, frases_doenca):
    for tipos in list(frases_doenca):
        for f in frases_doenca[tipos]:
            if((type(f) is not float) and (type(txt.CONTEUDO) is not float)):
                res = fuzz.partial_ratio(f, txt.CONTEUDO.lower())
                if res >= 90:
                    return(tipos, res, txt)

2497215


In [6]:
import time
start_time = time.time()
results = Parallel(n_jobs=num_cores)(delayed(checa_texto)(index, txt, frases_doenca) for index, txt in dado_completo.iterrows())
print(time.time() - start_time)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pos/derickmath/miniconda3/lib/python3.7/site-packages/joblib/parallel.py", line 833, in retrieve
    self._output.extend(job.get(timeout=self.timeout))
  File "/home/pos/derickmath/miniconda3/lib/python3.7/site-packages/joblib/_parallel_backends.py", line 521, in wrap_future_result
    return future.result(timeout=timeout)
  File "/home/pos/derickmath/miniconda3/lib/python3.7/concurrent/futures/_base.py", line 427, in result
    self._condition.wait(timeout)
  File "/home/pos/derickmath/miniconda3/lib/python3.7/threading.py", line 296, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pos/derickmath/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-9d5826949c86>", line 3, in <module>
    results = Pa

KeyboardInterrupt: 

In [75]:
i = 0
for r in results:
    if r != None:
        i+=1    
print(i)

216805


In [92]:
result_distancia = pd.DataFrame([x for x in results if x != None])
result_distancia.to_csv("resultado_all_pathologys")
#result_distancia.to_csv("resultado_interferencia_distancia")

In [106]:
pd.options.display.max_colwidth = 5000

print(results)
dado_completo[dado_completo["ID_EXAME"] == 2092328]["CONTEUDO"]

KeyboardInterrupt: 

First Classifier

In [20]:
lstm_input_data = []
print (lstm_input_data)

[]


Execução LSTM

In [67]:
output_file = "saida_lstm_model_10"

class_dim = 3
k = 10

#number of iteractions 
epochs = 10 
batch_size = 256
training_size = 1
#max number of words in embedding
embed_dim = 512
lstm_out_dim = 256
top_words = 7500

In [68]:
sentiment = []
text = []
lstm_input = []
####
file_out = open(output_file, 'w')
######

with open(dado_entrada_lstm_c) as file:
    lstm_input_data1 = [x.split("\t") for x in file.read().split("\n")]
size1 = len(lstm_input_data1)
with open(dado_entrada_lstm_e) as file:
    lstm_input_data2 = next_n_lines(file, size1)
    
size2 = len(lstm_input_data2)
lstm_input_data = lstm_input_data1 + lstm_input_data2
size = len(lstm_input_data)
print(size1, size2, size)

1 1 2


In [51]:
for i in range(size):
    lstm_input.append([ lstm_input_data[i][0] , re.sub(' +',' ',re.sub("[^a-zA-Z 0-9;á-Ź]+", " ", str(lstm_input_data[i]))).split(";")])
    sentiment.append(lstm_input[i][0])
    text.append(lstm_input[i][1])
tokenizer = Tokenizer(num_words = top_words, split = ' ')
tokenizer.fit_on_texts(text)
X = tokenizer.texts_to_sequences(text)
X_train = X
Y_train = sentiment
folds = list(StratifiedKFold(n_splits=k, shuffle=True, random_state=1).split(X, sentiment))

/home/disk2/speed/derickmath/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:626: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [52]:
shape = pad_sequences(X).shape[1]
model = Sequential()
model.add(Embedding(top_words, embed_dim,input_length = shape))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out_dim, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(class_dim, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 8, 512)            3840000   
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 8, 512)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 771       
Total params: 4,628,227
Trainable params: 4,628,227
Non-trainable params: 0
_________________________________________________________________
None


In [53]:
X = pad_sequences(X)
Y = pd.get_dummies(sentiment).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = (1-training_size), random_state = 42)
model.fit(X_train, Y_train, epochs = epochs, batch_size=batch_size, verbose = 2)

#model.save(output_file)

Epoch 1/25
 - 20s - loss: 0.3712 - acc: 0.8329
Epoch 2/25
 - 19s - loss: 0.1501 - acc: 0.9412
Epoch 3/25
 - 19s - loss: 0.1322 - acc: 0.9463
Epoch 4/25
 - 19s - loss: 0.1305 - acc: 0.9464
Epoch 5/25
 - 19s - loss: 0.1300 - acc: 0.9467
Epoch 6/25
 - 19s - loss: 0.1282 - acc: 0.9467
Epoch 7/25
 - 19s - loss: 0.1281 - acc: 0.9466
Epoch 8/25
 - 19s - loss: 0.1277 - acc: 0.9464
Epoch 9/25
 - 19s - loss: 0.1267 - acc: 0.9466
Epoch 10/25
 - 18s - loss: 0.1270 - acc: 0.9466
Epoch 11/25
 - 18s - loss: 0.1265 - acc: 0.9467
Epoch 12/25
 - 19s - loss: 0.1268 - acc: 0.9466
Epoch 13/25
 - 19s - loss: 0.1269 - acc: 0.9467
Epoch 14/25
 - 19s - loss: 0.1268 - acc: 0.9467
Epoch 15/25
 - 18s - loss: 0.1264 - acc: 0.9467
Epoch 16/25
 - 19s - loss: 0.1268 - acc: 0.9466
Epoch 17/25
 - 19s - loss: 0.1262 - acc: 0.9466
Epoch 18/25
 - 19s - loss: 0.1261 - acc: 0.9468
Epoch 19/25
 - 19s - loss: 0.1259 - acc: 0.9466
Epoch 20/25
 - 19s - loss: 0.1263 - acc: 0.9467
Epoch 21/25
 - 19s - loss: 0.1263 - acc: 0.9466
E

In [54]:
with open(dado_saida_d) as file:
    lstm_teste = [x.split("\t") for x in file.read().split("\n")]
size = len(lstm_teste)
sentiment_test = []
text_test = []
lstm_input_test = []
for i in range(size - 1):
    lstm_input_test.append([ lstm_teste[i][0] , re.sub(' +',' ',re.sub("[^a-zA-Z 0-9;á-Ź]+", " ", str(lstm_teste[i]))).split(";")])
    sentiment_test.append(lstm_input_test[i][0])
    text_test.append(lstm_input_test[i][1])

#tokenizer.fit_on_texts(text_test)
X = tokenizer.texts_to_sequences(text_test)
X = pad_sequences(X, maxlen=12)
X_test = X
Y_test = sentiment_test


In [56]:
result = model.predict(X_test)
print(X_test)
#print(len(text_test))
#np.savetxt('saida_lstm_result_10', model.predict_classes(X_test))

ValueError: Error when checking input: expected embedding_3_input to have shape (8,) but got array with shape (12,)